In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain_qdrant import Qdrant

huggingFaceApiToken = os.environ["HUGGINGFACEHUB_API_TOKEN"]
qdrant_url = os.environ["QDRANT_URL"]
# embedding
embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingFaceApiToken, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

# LLM
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token=huggingFaceApiToken,
)

# Retriever
store = Qdrant.from_existing_collection(
    embedding=embedding,
    collection_name="lilianweng_agent",
    url=qdrant_url,
)
retriever = store.as_retriever(search_kwargs={"k": 1})


def printEverything(everything, name=""):
    print(f"{name} {everything}")
    return everything

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs.extend(loader.load())

In [ ]:
docs

In [ ]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chain = (
    # {"doc": lambda x: {print(x), x.page_content }} | 
    ChatPromptTemplate.from_template("Citations are marked as [number]. Give me a summary without the citations:\n\n{doc}")
    | llm
    | printEverything
    | StrOutputParser()
)

summaries = chain.invoke(docs)

In [ ]:
summaries

In [ ]:
from email.policy import HTTP
from langchain.storage import InMemoryByteStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from qdrant_client import QdrantClient

# The vectorstore to use to index the child chunks
qdrant_url = os.environ["QDRANT_URL"]
vectorstore = Qdrant(collection_name="summaries", embeddings=embedding, client=QdrantClient(url=qdrant_url))
# vectorstore = Chroma(collection_name="summaries", embedding_function=embedding)


# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]


# Add
# # run this to create collection in qdrant
# store = Qdrant.from_documents(
#     summary_docs,
#     embedding,
#     url=qdrant_url,
#     prefer_grpc=True,
#     collection_name="summaries",
# )
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

In [ ]:
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]